In [2]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch, PyTorchModel
import datetime
import json

import torch
import argparse
import os
from typing import Hashable
from pandas.core.series import Series
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchvision.ops import StochasticDepth
from torch.cuda.amp import GradScaler, autocast
import sys
from torch.optim.lr_scheduler import CosineAnnealingLR
# import pickle
# from io import BytesIO
# import boto3
import timm


import copy
import json
import gc


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
    log_loss,
    hinge_loss)

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn import tree 
from sklearn.svm import SVC


from torchvision.models import resnet18, ResNet18_Weights, swin_b
import torch

import torchvision
from torchvision.models import Swin_T_Weights, ResNet18_Weights, Swin_S_Weights, Swin_B_Weights
import torchvision.models as models
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights, ResNet50_Weights

import torchvision.models as models
import os
import psutil

# import cv2

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns
import os

from PIL import Image
from tqdm import tqdm

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/amannindra/Library/Application Support/sagemaker/config.yaml


/Users/amannindra/Projects/DSS-Image-Classification/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class AnimalDatasetSwinTest(Dataset):
    def __init__(self, dataframe, transform=None, folder="", img_size=224):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform
        self.img_size = img_size
        self.folder = folder
        label_columns = [
            "antelope_duiker",
            "bird",
            "blank",
            "civet_genet",
            "hog",
            "leopard",
            "monkey_prosimian",
            "rodent",
        ]
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        filename = self.dataframe.iloc[idx]["filepath"]
        image_path = os.path.join(self.folder, filename)
        try:
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image file not found: {image_path}")
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Error loading {image_path}: {e}")
            image = Image.new('RGB', (self.img_size, self.img_size), color=(128, 128, 128))
        if self.transform:
            image = self.transform(image)
        return image, self.dataframe.iloc[idx]["id"]
    


In [4]:
img_size = 224
test_transform = transforms.Compose(
        [
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

In [5]:

num_classes = 8
model_name = 'convnext_large.fb_in22k_ft_in1k'
model = timm.create_model(model_name, pretrained=False, num_classes=num_classes) 
wieghts_path = "convnext-1/models/convnext-largeWeights_3_best.pth"
state_dict = torch.load(wieghts_path, map_location=torch.device('cpu')) 
model.load_state_dict(state_dict['model'] if 'model' in state_dict else state_dict)
print(model)
print(f"Successfully loaded fine-tuned weights from {wieghts_path}")
# model.head = nn.Linear(model.head.in_features, num_classes)
# model.load_state_dict(torch.load("swinb-final3/models/swinb-final-3Weights_8_best.pth", map_location="cpu"))
# model.eval()

base_path = "data"
test_folder = os.path.join(base_path, "test_features")
test_features_csv = os.path.join(base_path, "test_features.csv")
print(f"Test folder: {test_folder}")
if not os.path.exists(test_folder):
    raise FileNotFoundError(f"Test folder not found: {test_folder}")

test_df = pd.read_csv(os.path.join(base_path, "test_features.csv"))

test_df

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (norm): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=768, out_features=192, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), g

,id,filepath,site
0,ZJ016488,test_features/ZJ016488.jpg,S0082
1,ZJ016489,test_features/ZJ016489.jpg,S0040
2,ZJ016490,test_features/ZJ016490.jpg,S0040
3,ZJ016491,test_features/ZJ016491.jpg,S0041
4,ZJ016492,test_features/ZJ016492.jpg,S0040
...,...,...,...
4459,ZJ020947,test_features/ZJ020947.jpg,S0086
4460,ZJ020948,test_features/ZJ020948.jpg,S0152
4461,ZJ020949,test_features/ZJ020949.jpg,S0162
4462,ZJ020950,test_features/ZJ020950.jpg,S0055


In [8]:
test_dataset = AnimalDatasetSwinTest(test_df, transform=test_transform, folder=base_path, img_size=img_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

keys = ["antelope_duiker", "bird", "blank", "civet_genet", "hog", "leopard", "monkey_prosimian", "rodent"]

submission_df = pd.DataFrame(columns=["id", "antelope_duiker", "bird", "blank", "civet_genet", "hog", "leopard", "monkey_prosimian", "rodent"])
rows = []


pbar = tqdm(test_loader, total=len(test_loader))
for batch_idx, (images, ids) in enumerate(pbar):
    images = images.to(device)

    with torch.no_grad():
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)

    probs = probs.cpu().numpy()

    for i in range(len(ids)):
        rows.append(
            [ids[i], *probs[i]]
        )
    if batch_idx % 10 == 0:
        print(f"Batch {batch_idx} completed")
    
    
    # for i in range(len(ids)):
    #     print(f"ID: {ids[i]}, Predicted: {preds[i].item()}")
submission_df = pd.DataFrame(
    rows,
    columns=["id"] + keys
)

  1%|          | 1/140 [00:05<13:37,  5.88s/it]

Batch 0 completed


  8%|▊         | 11/140 [01:23<16:07,  7.50s/it]

Batch 10 completed


 15%|█▌        | 21/140 [02:20<11:25,  5.76s/it]

...

 86%|████████▋ | 121/140 [19:57<04:17, 13.54s/it]   

Batch 120 completed


 94%|█████████▎| 131/140 [21:37<00:58,  6.53s/it]

Batch 130 completed


100%|██████████| 140/140 [22:25<00:00,  9.61s/it]


In [9]:
print(f"len(submission_df): {len(submission_df)}")
submission_df.head(5)

len(submission_df): 4464


,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
0,ZJ016488,0.025933,0.000176,0.200344,0.425523,0.000465,0.009403,0.000740,0.337416
1,ZJ016489,0.139847,0.091046,0.626210,0.011904,0.016749,0.003262,0.010855,0.100125
2,ZJ016490,0.041336,0.012198,0.288875,0.089026,0.344301,0.004323,0.001714,0.218227
3,ZJ016491,0.000011,0.000022,0.000297,0.000055,0.000030,0.999566,0.000008,0.000010
4,ZJ016492,0.009122,0.003619,0.056968,0.018276,0.001804,0.000129,0.750204,0.159877


In [10]:
submission_df.to_csv("final_convnext_large_csv.csv", index=False)

In [15]:
from urllib.request import urlopen
from PIL import Image
import timm

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model('convnext_large.fb_in22k', pretrained=True)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

/Users/amannindra/Projects/DSS-Image-Classification/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
transforms

Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)